### Assignment 2 - To Embed or Not to Embed

All text from assignment template deleted to make the notebook easier to navigate


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
%matplotlib inline

torch.manual_seed(1)

In [3]:
#Mount GDrive to save models
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
#Load data
!gdown 1B26XNKmZ6wS1tFR5NN3eWYBZlmnZC5gH
!gdown 1vLV8H4LnT362p77v2cgLVfN5NIeDbGwx


import pandas as pd
tripadvisor=pd.read_csv('tripadvisor_hotel_reviews.csv')

with open(f'scifi.txt')as f:
    scify = f.read()

Downloading...
From: https://drive.google.com/uc?id=1B26XNKmZ6wS1tFR5NN3eWYBZlmnZC5gH
To: /content/tripadvisor_hotel_reviews.csv
100% 15.0M/15.0M [00:00<00:00, 131MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vLV8H4LnT362p77v2cgLVfN5NIeDbGwx
To: /content/scifi.txt
100% 86.1M/86.1M [00:00<00:00, 121MB/s]


## Step 1: making example work

In [25]:
# Working version for the
import os
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

nltk.download('stopwords')
model_save_name="test_model_mounted.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
model_path = os.path.join(os.getcwd(), path)

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 10

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells""".lower().split()

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


stopwords_english = stopwords.words('english')

def remove_stopwords(text):
    new_text = []
    for word in text:
        if word in stopwords_english:
            continue
        else:
            new_text.append(word)

    return new_text


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text

text=remove_stopwords(raw_text)
text=perform_stemming(text)


# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(text) - 2):
    context = [text[i - 2], text[i - 1],
               text[i + 1], text[i + 2]]
    target = text[i]
    data.append((context, target))



class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))
        out = self.linear1(embeds)
        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# create your model and train.  here are some functions to help you make
# the data ready for use by your module
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.005)



for epoch in range(20):
    total_loss = 0
    for context, target in data:

        # Prepare input vector
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # zero out gradients
        model.zero_grad()

        # run forward pass
        log_probs = model(context_idxs)

        # Compute loss function
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

torch.save(model.state_dict(), model_path)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[99.98636531829834, 98.96921420097351, 97.9815821647644, 97.02028155326843, 96.08341884613037, 95.16914963722229, 94.27534770965576, 93.40070271492004, 92.54347133636475, 91.70263171195984, 90.87639713287354, 90.06389236450195, 89.26329183578491, 88.47424221038818, 87.69621682167053, 86.92822360992432, 86.16837477684021, 85.41659188270569, 84.67174315452576, 83.9335618019104]


## Tripadvisor embedding CBOW2

Embedd the words in the Hotel Review dataset with a context size of 2



In [58]:
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader
nltk.download('stopwords')

import string
exclude = string.punctuation

#Pre-processing
def remove_punc(text):
    translation_table = str.maketrans(exclude, ' ' * len(exclude))
    return text.translate(translation_table)



def is_english_word(word):
    from nltk.corpus import words

    # Make sure the word is in lowercase
    word = word.lower()

    # Check if the word exists in the English words corpus
    return word in words.words()

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


stopwords_english = stopwords.words('english')

def remove_stopwords(text):
    new_text = []
    for word in text:
        if word in stopwords_english:
            continue
        else:
            new_text.append(word)

    return new_text


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text


data = []

#Loop through dataset
for raw_text in tripadvisor.Review:
    text_without_punc = remove_punc(raw_text)


    split_by_periods = text_without_punc.split(' ')
    # Split each resulting substring along whitespaces and "/"
    result = [substring.split() for substring in split_by_periods]


    # Flatten the list of lists to get a single list of words and phrases
    flat_result = [word for sublist in result for word in sublist]

    #drop all entries that contain numbers -> mostly times, dates, room number..
    filtered_result = [word for word in flat_result if not any(char.isnumeric() for char in word)]

    text_no_stop=remove_stopwords(filtered_result)
   #text_stemmed=perform_stemming(text_no_stop)


    # Remove all words that are not in english dictionary - removed due to very long runtime
    #english_words = [word for word in text_no_stop if is_english_word(word)]

    #text=english_words

    text=text_no_stop



    #Get contexts and targets
    for i in range(2, len(text) - 2):
        context = [text[i - 2], text[i - 1],
                  text[i + 1], text[i + 2]]
        target = text[i]
        data.append((context, target))


unique_words = set()

for item in data:
    words, _ = item
    unique_words.update(words)

# Convert the set to a list to display the unique vocabulary
vocab = unique_words
vocab_size = len(vocab)



def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)



word_to_ix = {word: i for i, word in enumerate(vocab)}

#Transform data to numeric
X, y = [], []
for context, target in data:
  X.append(context)
  y.append(target)


X_numeric=[]
y_numeric=[]
for i in X:
  X_numeric.append([word_to_ix[w] for w in i])
print(X_numeric)
X_train=torch.FloatTensor(X_numeric)

y_numeric=[]
for i in y:
  y_numeric.append(word_to_ix[i])
print(y_numeric)
y_train=torch.FloatTensor(y_numeric)




num_batches=12000

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load data to dataloader
class MyDataset(IterableDataset):
    def __init__(self, data_X, data_y):
        assert len(data_X) == len(data_y)
        self.data_X = data_X.to(device)
        self.data_y = data_y.to(device)

    def __len__(self):
        return len(self.data_X)

    def __iter__(self):
        for i in range(len(self.data_X)):
            yield (self.data_X[i], self.data_y[i])

train_set = MyDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=num_batches)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
#Train the model
import os
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader


num_epochs=15
# Source: https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 50

model_save_name="tripadvisor_CBOW2.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
model_path = os.path.join(os.getcwd(), path)


class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))

        embeds = self.embeddings(inputs).sum(dim=1)

        out = self.linear1(embeds)
        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# create your model and train.  here are some functions to help you make
# the data ready for use by your module
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005)



for epoch in range(num_epochs):
    total_loss = 0
    counter=0
    for batch in train_loader:
        counter=counter+1

        context=batch[0]
        target=batch[1]
        counter=counter+1


        # Prepare input vector
        context_idxs = torch.tensor(context, dtype=torch.long).to(device)

        # zero out gradients
        model.zero_grad()

        # run forward pass
        log_probs = model(context_idxs)

        # Compute loss function
        loss = loss_function(log_probs, torch.tensor(target, dtype=torch.long).to(device))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()


        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], loss: {total_loss}]')
print(losses)

torch.save(model.state_dict(), path)



<ipython-input-60-259aea352da9>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context_idxs = torch.tensor(context, dtype=torch.long).to(device)
<ipython-input-60-259aea352da9>:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_function(log_probs, torch.tensor(target, dtype=torch.long).to(device))


Epoch [1/15], loss: 1772.2735004425049]
Epoch [2/15], loss: 1758.4999828338623]
Epoch [3/15], loss: 1744.860345840454]
Epoch [4/15], loss: 1731.2777700424194]
Epoch [5/15], loss: 1717.7436351776123]
Epoch [6/15], loss: 1704.4070415496826]
Epoch [7/15], loss: 1691.6781911849976]
Epoch [8/15], loss: 1680.0424394607544]
Epoch [9/15], loss: 1669.524401664734]
Epoch [10/15], loss: 1659.7103281021118]
Epoch [11/15], loss: 1650.2076044082642]
Epoch [12/15], loss: 1640.8156633377075]
Epoch [13/15], loss: 1631.5071926116943]
Epoch [14/15], loss: 1622.356206893921]
Epoch [15/15], loss: 1613.4701385498047]
[1772.2735004425049, 1758.4999828338623, 1744.860345840454, 1731.2777700424194, 1717.7436351776123, 1704.4070415496826, 1691.6781911849976, 1680.0424394607544, 1669.524401664734, 1659.7103281021118, 1650.2076044082642, 1640.8156633377075, 1631.5071926116943, 1622.356206893921, 1613.4701385498047]


**Make Predictions**
To check whether the embedding makes sense, we test it on some self-defined context words



In [28]:
import os
import torch
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

model_save_name="tripadvisor_CBOW2.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))
        print(inputs.shape)
        embeds = self.embeddings(inputs).sum(dim=0)

        out = self.linear1(embeds)

        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# Load the trained model

model_path = os.path.join(path)  # Replace with the path to your saved model
model = CBOW(vocab_size, 50)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()  # Set the model to evaluation mode

# Function to predict the next word
def predict_next_word(context, model):
    context_tokens = word_tokenize(context)  # Tokenize the context
    if len(context_tokens) < 2 * CONTEXT_SIZE:
        return "Context is too short"

    # Extract the context window of size CONTEXT_SIZE
    context_window = context_tokens[-CONTEXT_SIZE:] + context_tokens[:CONTEXT_SIZE]

    # Convert context to a list of word indices
    context_indices = [word_to_ix[word] for word in context_window]

    # Prepare input tensor
    context_tensor = torch.tensor(context_indices, dtype=torch.long).to(device)


    # Run forward pass
    with torch.no_grad():
        log_probs = model(context_tensor)

    # Get the index of the most likely word
    predicted_word_index = torch.argmax(log_probs).item()


    # Map the index back to the word
    predicted_word = ix_to_word[predicted_word_index]

    return predicted_word


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text

# Example usage
for context in ["view bath great beach", "food bad disappointing waiter","beach swim bad cold", "science running writing helping","bed quality sleep dream","pizza pasta fries potato"]:


  context=perform_stemming(context.split(" "))
  stemmed_context=""

  for x in context:
    stemmed_context+=x+" "
  stemmed_context



  predicted_word = predict_next_word(stemmed_context, model)
  print(f'Context: {context}, Predicted word: {predicted_word}')

torch.Size([4])
Context: ['view', 'bath', 'great', 'beach'], Predicted word: hotel
torch.Size([4])
Context: ['food', 'bad', 'disappoint', 'waiter'], Predicted word: hotel
torch.Size([4])
Context: ['beach', 'swim', 'bad', 'cold'], Predicted word: room
torch.Size([4])
Context: ['scienc', 'run', 'write', 'help'], Predicted word: hotel
torch.Size([4])
Context: ['bed', 'qualiti', 'sleep', 'dream'], Predicted word: hotel
torch.Size([4])
Context: ['pizza', 'pasta', 'fri', 'potato'], Predicted word: room


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Tripadvisor CBOW5

In [21]:
#Pre-process and load to dataloader
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader
nltk.download('stopwords')

import string
exclude = string.punctuation

#nltk.download('words')
def remove_punc(text):
    translation_table = str.maketrans(exclude, ' ' * len(exclude))
    return text.translate(translation_table)



def is_english_word(word):
    from nltk.corpus import words

    # Make sure the word is in lowercase
    word = word.lower()

    # Check if the word exists in the English words corpus
    return word in words.words()

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


stopwords_english = stopwords.words('english')

def remove_stopwords(text):
    new_text = []
    for word in text:
        if word in stopwords_english:
            continue
        else:
            new_text.append(word)

    return new_text


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text


data = []

#Loop through dataset
for raw_text in tripadvisor.Review:
    text_without_punc = remove_punc(raw_text)

    #split text to list
    split_by_periods = text_without_punc.split(' ')
    # Split each resulting substring along whitespaces and "/"
    result = [substring.split() for substring in split_by_periods]


    # Flatten the list of lists to get a single list of words and phrases
    flat_result = [word for sublist in result for word in sublist]

    #drop all entries that contain numbers -> mostly times, dates, room number..
    filtered_result = [word for word in flat_result if not any(char.isnumeric() for char in word)]

    text_no_stop=remove_stopwords(filtered_result)
    #text_stemmed=perform_stemming(text_no_stop)


    # Remove all words that are not in english dictionary - removed due to very long runtime
    #english_words = [word for word in text_no_stop if is_english_word(word)]

    #text=english_words

    text=text_no_stop



    for i in range(5, len(text) - 5):
        context = [text[i - 5],
                  text[i - 4], text[i - 3],
                  text[i - 2], text[i - 1],
                  text[i + 1], text[i + 2],
                  text[i + 3], text[i +4],
                  text[i+5]]
        target = text[i]
        data.append((context, target))


unique_words = set()

for item in data:
    words, _ = item
    unique_words.update(words)

# Convert the set to a list to display the unique vocabulary
vocab = unique_words
vocab_size = len(vocab)



def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)



word_to_ix = {word: i for i, word in enumerate(vocab)}


X, y = [], []
for context, target in data:
  X.append(context)
  y.append(target)


X_numeric=[]
y_numeric=[]
for i in X:
  X_numeric.append([word_to_ix[w] for w in i])
print(X_numeric)
X_train=torch.FloatTensor(X_numeric)

y_numeric=[]
for i in y:
  y_numeric.append(word_to_ix[i])
print(y_numeric)
y_train=torch.FloatTensor(y_numeric)




num_batches=6000

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class MyDataset(IterableDataset):
    def __init__(self, data_X, data_y):
        assert len(data_X) == len(data_y)
        self.data_X = data_X.to(device)
        self.data_y = data_y.to(device)

    def __len__(self):
        return len(self.data_X)

    def __iter__(self):
        for i in range(len(self.data_X)):
            yield (self.data_X[i], self.data_y[i])

train_set = MyDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=num_batches)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader

num_epochs=15

# Source: https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py
CONTEXT_SIZE = 5 # 5 words to the left, 2 to the right
EMBEDDING_DIM = 50

model_save_name="tripadvisor_CBOW5.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
model_path = os.path.join(os.getcwd(), path)


class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))

        embeds = self.embeddings(inputs).sum(dim=1)

        out = self.linear1(embeds)
        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# create your model and train.  here are some functions to help you make
# the data ready for use by your module
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005)



for epoch in range(num_epochs):
    total_loss = 0
    counter=0
    for batch in train_loader:
        counter=counter+1

        context=batch[0]
        target=batch[1]
        counter=counter+1


        # Prepare input vector
        context_idxs = torch.tensor(context, dtype=torch.long).to(device)

        # zero out gradients
        model.zero_grad()

        # run forward pass
        log_probs = model(context_idxs)

        # Compute loss function
        loss = loss_function(log_probs, torch.tensor(target, dtype=torch.long).to(device))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()


        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], loss: {total_loss}]')
print(losses)

torch.save(model.state_dict(), path)


<ipython-input-31-59cf360a8157>:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context_idxs = torch.tensor(context, dtype=torch.long).to(device)
<ipython-input-31-59cf360a8157>:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_function(log_probs, torch.tensor(target, dtype=torch.long).to(device))


Epoch [1/15], loss: 3208.8699626922607]
Epoch [2/15], loss: 3066.107057571411]
Epoch [3/15], loss: 2968.185814857483]
Epoch [4/15], loss: 2888.826210975647]
Epoch [5/15], loss: 2823.719162940979]
Epoch [6/15], loss: 2771.010648727417]
Epoch [7/15], loss: 2727.4765796661377]
Epoch [8/15], loss: 2690.543646812439]
Epoch [9/15], loss: 2658.7761278152466]
Epoch [10/15], loss: 2631.071268081665]
Epoch [11/15], loss: 2606.6073932647705]
Epoch [12/15], loss: 2584.7755184173584]
Epoch [13/15], loss: 2565.126700401306]
Epoch [14/15], loss: 2547.345413684845]
Epoch [15/15], loss: 2531.1996302604675]
[3208.8699626922607, 3066.107057571411, 2968.185814857483, 2888.826210975647, 2823.719162940979, 2771.010648727417, 2727.4765796661377, 2690.543646812439, 2658.7761278152466, 2631.071268081665, 2606.6073932647705, 2584.7755184173584, 2565.126700401306, 2547.345413684845, 2531.1996302604675]


**Make Predictions**
To check whether the embedding makes sense, we test it on some self-defined context words



In [25]:
import os
import torch
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

model_save_name="tripadvisor_CBOW5.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))
        print(inputs.shape)
        embeds = self.embeddings(inputs).sum(dim=0)

        out = self.linear1(embeds)

        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# Load the trained model

model_path = os.path.join(path)  # Replace with the path to your saved model
model = CBOW(vocab_size, 50)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()  # Set the model to evaluation mode

# Function to predict the next word
def predict_next_word(context, model):
    context_tokens = word_tokenize(context)  # Tokenize the context
    if len(context_tokens) < 2 * CONTEXT_SIZE:
        return "Context is too short"

    # Extract the context window of size CONTEXT_SIZE
    context_window = context_tokens[-CONTEXT_SIZE:] + context_tokens[:CONTEXT_SIZE]

    # Convert context to a list of word indices
    context_indices = [word_to_ix[word] for word in context_window]

    # Prepare input tensor
    context_tensor = torch.tensor(context_indices, dtype=torch.long).to(device)


    # Run forward pass
    with torch.no_grad():
        log_probs = model(context_tensor)

    # Get the index of the most likely word
    predicted_word_index = torch.argmax(log_probs).item()


    # Map the index back to the word
    predicted_word = ix_to_word[predicted_word_index]

    return predicted_word


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text

# Example usage
for context in ["view bath great beach swimming house lake water warm great", "pasta pizza seafood good potato fries rice tomato view table", "hard sleep uncomfortable loud blanket pillow bad noisy good tired", "said reception check-in late fast good complicated wait lovely experience room"]:


  context=perform_stemming(context.split(" "))
  stemmed_context=""

  for x in context:
    stemmed_context+=x+" "
  stemmed_context



  predicted_word = predict_next_word(stemmed_context, model)
  print(f'Context: {context}, Predicted word: {predicted_word}')

torch.Size([4])
Context: ['view', 'bath', 'great', 'beach', 'swim', 'hous', 'lake', 'water', 'warm', 'great'], Predicted word: beachrestaraunt
torch.Size([4])
Context: ['pasta', 'pizza', 'seafood', 'good', 'potato', 'fri', 'rice', 'tomato', 'view', 'tabl'], Predicted word: mirador
torch.Size([4])
Context: ['hard', 'sleep', 'uncomfort', 'loud', 'blanket', 'pillow', 'bad', 'noisi', 'good', 'tire'], Predicted word: beachrestaraunt
torch.Size([4])
Context: ['said', 'recept', 'check-in', 'late', 'fast', 'good', 'complic', 'wait', 'love', 'experi', 'room'], Predicted word: mirador


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Scifiy Text

In [5]:
#Preprocess data and load to dataloader
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader
nltk.download('stopwords')

import string
exclude = string.punctuation

def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))


#nltk.download('words')




def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


stopwords_english = stopwords.words('english')

def remove_stopwords(text):
    new_text = []
    for word in text:
        if word in stopwords_english:
            continue
        else:
            new_text.append(word)

    return new_text


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text


data = []

raw_text=scify.lower()

text_without_punc = remove_punc(raw_text)

chunk_size = 500

chunks = [text_without_punc[i:i+chunk_size] for i in range(0, len(text_without_punc), chunk_size)]


print(text_without_punc[:400])

#
#split_by_periods = raw_text.split('.')


# Split each resulting substring along whitespaces and "/"
result = [substring.split() for substring in chunks]


#Flatten the list of lists to get a single list of words and phrases
flat_result = [word for sublist in result for word in sublist]

#drop all entries that contain numbers -> mostly times, dates, room number..
filtered_result = [word for word in flat_result if not any(char.isnumeric() for char in word)]

text_no_stop=remove_stopwords(filtered_result)
#text_stemmed=perform_stemming(text_no_stop)


# Only keep word that exist in the english lanugage
#english_words = [word for word in text_no_stop if is_english_word(word)]

#text=english_words

text=text_no_stop






for i in range(2, len(text) - 2):
    context = [text[i - 2], text[i - 1],
              text[i + 1], text[i + 2]]
    target = text[i]
    data.append((context, target))


unique_words = set()

for item in data:
    words, _ = item
    unique_words.update(words)

# Convert the set to a list to display the unique vocabulary
vocab = unique_words
vocab_size = len(vocab)



def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)



word_to_ix = {word: i for i, word in enumerate(vocab)}




X, y = [], []
for context, target in data:
  X.append(context)
  y.append(target)


X_numeric=[]
y_numeric=[]
for i in X:
  X_numeric.append([word_to_ix[w] for w in i])
print(X_numeric)
X_train=torch.FloatTensor(X_numeric)

y_numeric=[]
for i in y:
  y_numeric.append(word_to_ix[i])
print(y_numeric)
y_train=torch.FloatTensor(y_numeric)




num_batches=3000

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class MyDataset(IterableDataset):
    def __init__(self, data_X, data_y):
        assert len(data_X) == len(data_y)
        self.data_X = data_X.to(device)
        self.data_y = data_y.to(device)

    def __len__(self):
        return len(self.data_X)

    def __iter__(self):
        for i in range(len(self.data_X)):
            yield (self.data_X[i], self.data_y[i])

train_set = MyDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=num_batches)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


march  all stories new and complete publisher editor if is published bimonthly by quinn publishing company inc kingston new york volume  no  copyright  by quinn publishing company inc application for entry as second class matter at post office buffalo new york pending subscription  for  issues in us and possessions canada  for  issues elsewhere  aiiow four weeks for change of address all stories a


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
#Train Model
import os
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader

num_epochs=3

# Source: https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 50

model_save_name="scify_CBOW2.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
model_path = os.path.join(os.getcwd(), path)


class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))

        embeds = self.embeddings(inputs).sum(dim=1)

        out = self.linear1(embeds)
        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# create your model and train.  here are some functions to help you make
# the data ready for use by your module
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005)



for epoch in range(num_epochs):
    total_loss = 0
    counter=0
    for batch in train_loader:
        counter=counter+1

        context=batch[0]
        target=batch[1]
        counter=counter+1


        # Prepare input vector
        context_idxs = torch.tensor(context, dtype=torch.long).to(device)

        # zero out gradients
        model.zero_grad()

        # run forward pass
        log_probs = model(context_idxs)

        # Compute loss function
        loss = loss_function(log_probs, torch.tensor(target, dtype=torch.long).to(device))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()


        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], loss: {total_loss}]')
print(losses)

torch.save(model.state_dict(), path)

<ipython-input-6-49434899bb0d>:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context_idxs = torch.tensor(context, dtype=torch.long).to(device)
<ipython-input-6-49434899bb0d>:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_function(log_probs, torch.tensor(target, dtype=torch.long).to(device))


Epoch [1/3], loss: 32425.964317321777]
Epoch [2/3], loss: 31331.134305000305]
Epoch [3/3], loss: 30287.25228023529]
[32425.964317321777, 31331.134305000305, 30287.25228023529]


**Make Predictions**
To check whether the embedding makes sense, we test it on some self-defined context words

In [20]:
import os
import torch
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

model_save_name="scify_CBOW2.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))
        print(inputs.shape)
        embeds = self.embeddings(inputs).sum(dim=0)

        out = self.linear1(embeds)

        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# Load the trained model

model_path = os.path.join(path)  # Replace with the path to your saved model
model = CBOW(vocab_size, 50)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()  # Set the model to evaluation mode

# Function to predict the next word
def predict_next_word(context, model):
    context_tokens = word_tokenize(context)  # Tokenize the context
    if len(context_tokens) < 2 * CONTEXT_SIZE:
        return "Context is too short"

    # Extract the context window of size CONTEXT_SIZE
    context_window = context_tokens[-CONTEXT_SIZE:] + context_tokens[:CONTEXT_SIZE]

    # Convert context to a list of word indices
    context_indices = [word_to_ix[word] for word in context_window]

    # Prepare input tensor
    context_tensor = torch.tensor(context_indices, dtype=torch.long).to(device)


    # Run forward pass
    with torch.no_grad():
        log_probs = model(context_tensor)

    # Get the index of the most likely word
    predicted_word_index = torch.argmax(log_probs).item()


    # Map the index back to the word
    predicted_word = ix_to_word[predicted_word_index]

    return predicted_word


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text

# Example usage
#for context in ["view bath great beach", "food bad disappointing waiter","beach swim bad cold", "science running writing helping","bed quality sleep dream","pizza pasta fries potato"]:
for context in ["flying alien rocket planet", "table food potato carrot", 'space sky star planet','assignment hard model run']:

  context=perform_stemming(context.split(" "))
  stemmed_context=""

  for x in context:
    stemmed_context+=x+" "
  stemmed_context



  predicted_word = predict_next_word(stemmed_context, model)
  print(f'Context: {context}, Predicted word: {predicted_word}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


torch.Size([4])
Context: ['fli', 'alien', 'rocket', 'planet'], Predicted word: like
torch.Size([4])
Context: ['tabl', 'food', 'potato', 'carrot'], Predicted word: said
torch.Size([4])
Context: ['space', 'sky', 'star', 'planet'], Predicted word: said
torch.Size([4])
Context: ['assign', 'hard', 'model', 'run'], Predicted word: said


## Closest neighbours Tripadvisor CBOW2

In [20]:
# Read and Preprocess (if this codeblock is run in isolation)
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader
nltk.download('stopwords')

import string
exclude = string.punctuation

#nltk.download('words')
def remove_punc(text):
    translation_table = str.maketrans(exclude, ' ' * len(exclude))
    return text.translate(translation_table)



def is_english_word(word):
    from nltk.corpus import words

    # Make sure the word is in lowercase
    word = word.lower()

    # Check if the word exists in the English words corpus
    return word in words.words()

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


stopwords_english = stopwords.words('english')

def remove_stopwords(text):
    new_text = []
    for word in text:
        if word in stopwords_english:
            continue
        else:
            new_text.append(word)

    return new_text


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text


data = []
for raw_text in tripadvisor.Review:
    text_without_punc = remove_punc(raw_text)


    split_by_periods = text_without_punc.split(' ')
    # Split each resulting substring along whitespaces and "/"
    result = [substring.split() for substring in split_by_periods]


    # Flatten the list of lists to get a single list of words and phrases
    flat_result = [word for sublist in result for word in sublist]

    #drop all entries that contain numbers -> mostly times, dates, room number..
    filtered_result = [word for word in flat_result if not any(char.isnumeric() for char in word)]

    text_no_stop=remove_stopwords(filtered_result)
    #text_stemmed=perform_stemming(text_no_stop)


    # Remove all words that are not in english dictionary - removed due to very long runtime
    #english_words = [word for word in text_no_stop if is_english_word(word)]

    #text=english_words

    text=text_no_stop



    for i in range(2, len(text) - 2):
        context = [text[i - 2], text[i - 1],
                  text[i + 1], text[i + 2]]
        target = text[i]
        data.append((context, target))


unique_words = set()

for item in data:
    words, _ = item
    unique_words.update(words)

# Convert the set to a list to display the unique vocabulary
vocab = unique_words
vocab_size = len(vocab)



def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)



word_to_ix = {word: i for i, word in enumerate(vocab)}


X, y = [], []
for context, target in data:
  X.append(context)
  y.append(target)


X_numeric=[]
y_numeric=[]
for i in X:
  X_numeric.append([word_to_ix[w] for w in i])
print(X_numeric)
X_train=torch.FloatTensor(X_numeric)

y_numeric=[]
for i in y:
  y_numeric.append(word_to_ix[i])
print(y_numeric)
y_train=torch.FloatTensor(y_numeric)




num_batches=12000

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class MyDataset(IterableDataset):
    def __init__(self, data_X, data_y):
        assert len(data_X) == len(data_y)
        self.data_X = data_X.to(device)
        self.data_y = data_y.to(device)

    def __len__(self):
        return len(self.data_X)

    def __iter__(self):
        for i in range(len(self.data_X)):
            yield (self.data_X[i], self.data_y[i])

train_set = MyDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=num_batches)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
# load model
ix_to_word = {ix:word for ix, word in enumerate(vocab)}
word_to_ix = {word:ix for ix, word in enumerate(vocab)}

model_save_name="tripadvisor_CBOW2.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))
        print(inputs.shape)
        embeds = self.embeddings(inputs).sum(dim=0)

        out = self.linear1(embeds)

        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# Load the trained model

model_path = os.path.join(path)  # Replace with the path to your saved model
model = CBOW(vocab_size, 50)
model.load_state_dict(torch.load(model_path))
model.to(device)

CBOW(
  (embeddings): Embedding(49093, 50)
  (linear1): Linear(in_features=50, out_features=200, bias=True)
  (linear2): Linear(in_features=200, out_features=49093, bias=True)
  (nonlin_1): ReLU()
  (nonlin_2): LogSoftmax(dim=-1)
)

In [22]:
#define closest neighbor
import torch.nn as nn
import torch

def get_closest_word(word,net, topn=5):
  word_distance=[]
  emb=net.embeddings
  pdist=nn.PairwiseDistance()
  i=word_to_ix[word]

  lookup_tensor_i=torch.tensor([i], dtype=torch.long).to(device)

  v_i =emb(lookup_tensor_i)

  for j in range(len(vocab)):
    if j != i:
      lookup_tensor_j=torch.tensor([j], dtype=torch.long).to(device)
      v_j = emb(lookup_tensor_j)
      word_distance.append((ix_to_word[j], float(pdist(v_i,v_j))))
  word_distance.sort(key=lambda x:x[1])
  return word_distance

In [75]:
# Get closest neighbors for 9 words
for i in ['staff', 'night', 'tableware','great','clean','immature', 'walk','like','appreciate']:
  print(i)
  print(get_closest_word(i, model)[:10])
  print('-----------------------------------')

staff
[('neck', 6.611093044281006), ('niche', 7.048586845397949), ('properly', 7.190108299255371), ('unroll', 7.377197265625), ('barbque', 7.398947238922119), ('corenr', 7.472538471221924), ('restrict', 7.5200347900390625), ('atmasphere', 7.5293474197387695), ('cliff', 7.629973888397217), ('candies', 7.659377574920654)]
-----------------------------------
night
[('disk', 6.037545204162598), ('charisma', 6.218534469604492), ('grotty', 6.5701704025268555), ('washroom', 6.7157392501831055), ('resrevations', 6.723171234130859), ('awfull', 6.727161407470703), ('jammed', 6.730695724487305), ('counterparts', 6.742620944976807), ('lacked', 6.764827251434326), ('vaca', 6.77960729598999)]
-----------------------------------
tableware
[('setups', 6.716906547546387), ('exercised', 6.754449367523193), ('spetacular', 6.83306360244751), ('staringout', 6.906892776489258), ('united', 6.92319917678833), ('acknowledging', 6.930212020874023), ('opening', 6.979665279388428), ('calls', 6.981204986572266), (

In [23]:
# Get closest neighbors for 1 words

for i in ['room', 'bed']:
  print(i)
  print(get_closest_word(i, model)[:10])
  print('-----------------------------------')

room
[('allcomers', 6.662619113922119), ('leidespein', 6.691622257232666), ('okey', 6.81837272644043), ('goodlooking', 6.867095470428467), ('cozza', 6.874899387359619), ('gentleman', 6.879683017730713), ('tui', 6.95367956161499), ('sherlock', 6.963742256164551), ('smells', 6.978347301483154), ('scrubbing', 7.022211074829102)]
-----------------------------------
bed
[('crystallline', 7.088943004608154), ('stevie', 7.132186412811279), ('filmed', 7.276320457458496), ('chow', 7.277835369110107), ('flow', 7.282066345214844), ('harrahs', 7.3771257400512695), ('mardis', 7.397796154022217), ('gentillesse', 7.400918006896973), ('dune', 7.474765777587891), ('compermise', 7.478332042694092)]
-----------------------------------


## Embedding Test Sci-fi

In [76]:
# Read and Preprocess (if this codeblock is run in isolation)
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from torch.utils.data import IterableDataset, DataLoader
nltk.download('stopwords')

import string
exclude = string.punctuation

def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))


#nltk.download('words')




def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


stopwords_english = stopwords.words('english')

def remove_stopwords(text):
    new_text = []
    for word in text:
        if word in stopwords_english:
            continue
        else:
            new_text.append(word)

    return new_text


ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return new_text


data = []

raw_text=scify.lower()

text_without_punc = remove_punc(raw_text)

chunk_size = 500

chunks = [text_without_punc[i:i+chunk_size] for i in range(0, len(text_without_punc), chunk_size)]


print(text_without_punc[:400])

#
#split_by_periods = raw_text.split('.')


# Split each resulting substring along whitespaces and "/"
result = [substring.split() for substring in chunks]


#Flatten the list of lists to get a single list of words and phrases
flat_result = [word for sublist in result for word in sublist]

#drop all entries that contain numbers -> mostly times, dates, room number..
filtered_result = [word for word in flat_result if not any(char.isnumeric() for char in word)]

text_no_stop=remove_stopwords(filtered_result)
text_stemmed=perform_stemming(text_no_stop)


# Only keep word that exist in the english lanugage
#english_words = [word for word in text_no_stop if is_english_word(word)]

#text=english_words

text=text_stemmed






for i in range(2, len(text) - 2):
    context = [text[i - 2], text[i - 1],
              text[i + 1], text[i + 2]]
    target = text[i]
    data.append((context, target))


unique_words = set()

for item in data:
    words, _ = item
    unique_words.update(words)

# Convert the set to a list to display the unique vocabulary
vocab = unique_words
vocab_size = len(vocab)



def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)



word_to_ix = {word: i for i, word in enumerate(vocab)}




X, y = [], []
for context, target in data:
  X.append(context)
  y.append(target)


X_numeric=[]
y_numeric=[]
for i in X:
  X_numeric.append([word_to_ix[w] for w in i])
print(X_numeric)
X_train=torch.FloatTensor(X_numeric)

y_numeric=[]
for i in y:
  y_numeric.append(word_to_ix[i])
print(y_numeric)
y_train=torch.FloatTensor(y_numeric)




num_batches=3000

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class MyDataset(IterableDataset):
    def __init__(self, data_X, data_y):
        assert len(data_X) == len(data_y)
        self.data_X = data_X.to(device)
        self.data_y = data_y.to(device)

    def __len__(self):
        return len(self.data_X)

    def __iter__(self):
        for i in range(len(self.data_X)):
            yield (self.data_X[i], self.data_y[i])

train_set = MyDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=num_batches)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


march  all stories new and complete publisher editor if is published bimonthly by quinn publishing company inc kingston new york volume  no  copyright  by quinn publishing company inc application for entry as second class matter at post office buffalo new york pending subscription  for  issues in us and possessions canada  for  issues elsewhere  aiiow four weeks for change of address all stories a


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
#load model
ix_to_word = {ix:word for ix, word in enumerate(vocab)}
word_to_ix = {word:ix for ix, word in enumerate(vocab)}

model_save_name="scify_CBOW2.pth"
path = f"/content/drive/MyDrive/'){model_save_name}"
class CBOW(nn.Module):

   def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 200)
        self.linear2 = nn.Linear(200, vocab_size)

        self.nonlin_1 = nn.ReLU()

        self.nonlin_2 = nn.LogSoftmax(dim = -1)


   def forward(self, inputs):
      # ask in the forum -> do we need to do the mean or sum
        #print(self.embeddings(inputs).shape)
        #embeds = torch.mean(self.embeddings(inputs), dim=0).view((1, -1))
        print(inputs.shape)
        embeds = self.embeddings(inputs).sum(dim=0)

        out = self.linear1(embeds)

        out = self.nonlin_1(out)
        out =  self.linear2(out)
        out = self.nonlin_2(out)

        return out

# Load the trained model

model_path = os.path.join(path)  # Replace with the path to your saved model
model = CBOW(vocab_size, 50)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()  # Set the model to evaluation mode

CBOW(
  (embeddings): Embedding(218662, 50)
  (linear1): Linear(in_features=50, out_features=200, bias=True)
  (linear2): Linear(in_features=200, out_features=218662, bias=True)
  (nonlin_1): ReLU()
  (nonlin_2): LogSoftmax(dim=-1)
)

In [16]:
# Get closest neighbors for 9 words
for i in ['time', 'room', 'dogfish','new','old', 'goddamned','take','fancies','examine']:
  print(i)
  print(get_closest_word(i, model)[:10])
  print('-----------------------------------')

time
[('profitable', 6.120347499847412), ('letsteve', 6.9447760581970215), ('ganymedeans', 6.966808319091797), ('smnething', 7.000274181365967), ('evhr', 7.019571304321289), ('projjose', 7.056467533111572), ('gondoliers', 7.0955610275268555), ('thass', 7.0956339836120605), ('spaceport', 7.123833656311035), ('rockmold', 7.132011890411377)]
-----------------------------------
room
[('paulines', 5.7859296798706055), ('artery', 5.852158069610596), ('energa', 5.993887424468994), ('barbar', 6.014967918395996), ('passra', 6.074892044067383), ('orbita', 6.076498508453369), ('ceentreels', 6.077091693878174), ('nny', 6.079671859741211), ('headaches', 6.112804889678955), ('therere', 6.123912334442139)]
-----------------------------------
dogfish
[('tribute', 6.2036261558532715), ('groundmoving', 6.207232475280762), ('milkless', 6.3910088539123535), ('hucinatory', 6.394628047943115), ('punchboards', 6.4385576248168945), ('svith', 6.442418098449707), ('mconfets', 6.467662334442139), ('cadmium', 6.5